# Problem 1
1. (50 points) Box Office Mojo (http://boxofficemojo.com) is a website that tracks a variety of information on movies released each year (such as genre, studio, box office revenues, etc.). You will write Python scripts that collect data on all the movies for the year that the user specifies and that are listed on the site. As an example, information on movies that were released in 2015 is available at: http://www.boxofficemojo.com/yearly/chart/?yr=2015&p=.htm
Collect the following information for each movie and save in a comma separated text file named movies.txt:
i. Movie Name.
ii. Movie ID. Note that this is not displayed on the webpage, but appears in the source code for the page. For example, for movie “Star Wars: The Force Awakens”, the Movie ID is “starwars7”.
iii. Studio that produced the movie.
iv. Total Gross Earnings
v. Total number of Theaters that the movie was shown
vi. Opening Gross Earnings
vii. Opening number of Theaters that the movie was shown
In addition answer the following questions:
i. What is the average Total Gross Earnings in the specified year
ii. Name the movie(s) that had the maximum number of theaters that they were shown in

# Algorithm
Step-1  Import beautifulsoup pakage, import request,urlopen, urllib.request, re, os 
Step-2  Ask user what year it is intrested in
Step-3  Enter site url for web scraping
Step-4  Define the count for movies, movies id, studio name, earnings and number of theaters
Step-5  Apply condition to find the rank of a movie
Step-6  Enter all the data collected from the site in a .txt file and and separate it using a ','
Step-7  Condition to open page for year entered buy the user and find necessary information
Step-8  Ask the year from the user and print the answer
Step-9  End

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

import re #import regular function
import os #import operating system dependent funtotionality

yearInst = "What is the year that you are interested in getting data:"
constPageHrefIncluded = "/yearly/chart/?page="
constFileName = "movies.txt"
constFileSavedNameInst = "The data is stored in file movies.txt"
# define what the program asks the user, where and by what name the files gets saved in text file 

urlBoxOfficeMojoHome = "http://www.boxofficemojo.com"
preUrlYear = "http://www.boxofficemojo.com/yearly/chart/?yr="
postUrlYear = "&p=.htm"
# enter site url and exact page for web scraping
_year = 0

globalMoviesWithMaxTheatres = 0
globalMaxMovieTheatreCount = 0
globalIsAverageEarningMovieFound = False
globalAverageTotalGrossEarningsForYear = ""
#define count for movies with max theatres, theatre count, gross earnings for the year
def removeChar(charToSub, a_strToStringChar):
  return re.sub(charToSub, '', a_strToStringChar)


def writeMoviesDataTable(a_strPageLink):
  currentPageSource = BeautifulSoup(urlopen(a_strPageLink).read())
#scrap page source of the page with the information for all the movies 
  allTablesList = currentPageSource.findAll('table')

  tableDataList = []
  currentTableIndex = 0
#Add content of data base in dictionary  
  isMovieTableFound = False
  for currentTable in allTablesList:
    tableDataList = currentTable.findAll('td')
#use find.all to find what you are exactly looking for    
    for currentTableData in tableDataList:
      if currentTableData.get_text() == "Rank":
        isMovieTableFound = True
        break
#condition to find the rank of the movie and close the loop
    if isMovieTableFound:
      break
    else:
      currentTableIndex += 1

  if isMovieTableFound == False:
      return
#condition to increase count for every data content found      
  movieTable = allTablesList[currentTableIndex]

  tableRows = movieTable.findAll('tr')
  currentRowIndex = 1
  rowCount = len(tableRows)

  while currentRowIndex < rowCount:
    currentRow = tableRows[currentRowIndex]
    columns = currentRow.findAll('td')
    columnCount = len(columns)

    if columnCount == 9 or columnCount == 8:
      rank = columns[0].get_text()
      title = columns[1]

      studio = removeChar(',',columns[2].get_text())
      totalGrossEarnings = removeChar(',',columns[3].get_text())

      totalTheatres = removeChar(',',columns[4].get_text())
# indentify that movie name, ID, studio name, gross earnings, number of total and opening theatres movie is shown all are separated by ',' in the text file
      global globalMaxMovieTheatreCount
      global globalMoviesWithMaxTheatres
      global globalIsAverageEarningMovieFound
      global globalAverageTotalGrossEarningsForYear

      if totalTheatres.isdigit():
        if int(totalTheatres) > globalMaxMovieTheatreCount:
          globalMaxMovieTheatreCount = int(totalTheatres)
          globalMoviesWithMaxTheatres = title.get_text()
        elif int(totalTheatres) == globalMaxMovieTheatreCount:
          globalMoviesWithMaxTheatres += "," + title.get_text()

      openingEarnings = removeChar(',',columns[5].get_text())

      openingTheatres = removeChar(',',columns[6].get_text())

      movieId = title.find('a').get('href')
      movieId = movieId.replace('.htm','')
      movieId = movieId.replace('/movies/?id=','')

      fileToWrite = open(constFileName, "a")
      fileToWrite.write(title.get_text() +"," + movieId +","+ studio +"," + totalGrossEarnings +","+ totalTheatres +","+ openingEarnings +","+ openingTheatres + "\n")
    elif globalIsAverageEarningMovieFound == False and globalAverageTotalGrossEarningsForYear == "" and columnCount == 7 or columnCount == 6:
      if columns[0].get_text() == "Averages:": 
        globalAverageTotalGrossEarningsForYear = columns[1].get_text()
        globalIsAverageEarningMovieFound = True
# indentify that movie name, ID, studio name, gross earnings, number of total and opening theatres movie are written in seperate columns
    currentRowIndex += 1



while True:
  currentSelection = input("\n"+yearInst)
  if currentSelection.isdigit():
    _year = int(currentSelection)  
  else:
    continue

  current_url = preUrlYear + str(_year) + postUrlYear
# condition to open page for the year entered by the user
  request = requests.get(current_url)

  if request.status_code != 200:
    print("Data " + current_url+" for year does not exist.")
    continue
# condition to ask question for the year again if the entered year does not exsist on the web page
  baseYearUrlToOpen = urlopen(current_url)
  readSource = BeautifulSoup(baseYearUrlToOpen.read())

  allPageList = []
  allPageSet = []

  allLinks = readSource.findAll('a')
#create a dictionary of page list and use find.all for finding exact data we nwwd

  for hrefTag in allLinks:
    if str(hrefTag.get('href')).find(constPageHrefIncluded) != -1:
      allPageList.append(urlBoxOfficeMojoHome + str(hrefTag.get('href')))
# use append to add data to the end of the list      
  allPageList.append(current_url)
  allPageSet = set(allPageList)

  if len(allPageSet) == 0:
    print("Data " + current_url+" for year does not exist.")
    continue

  if os.path.exists(constFileName):
    os.remove(constFileName)

  globalAverageTotalGrossEarningsForYear = ""
  globalMoviesWithMaxTheatres = ""
  globalIsAverageEarningMovieFound = False
  globalMaxMovieTheatreCount = 0

  for l_currentPage in allPageSet:
    writeMoviesDataTable(l_currentPage)
  print(constFileSavedNameInst)


  print("The average Total Gross Earnings for "+str(_year)+" is:" + globalAverageTotalGrossEarningsForYear)

  print("The movie(s) that were shown in the maximum number of theaters for "+str(_year)+" are:" + globalMoviesWithMaxTheatres)

# Problem 2
(50 points) The site Box Office Mojo (http://boxofficemojo.com) also has a People Index on movie actors (ordered by Total Gross) listed on the page: http://boxofficemojo.com/people/?view=Actor&pagenum=1&sort=sumgross&order=DESC&&p=.htm Collect the following information for each actor and save in a comma separated text file named actors.txt: i. Rank (Row number displayed on the page). ii. Actor Name. iii. Actor ID (it is in the source code of the webpage) iv. Total Gross v. Number of movies vi. # 1 picture In addition answer the following questions: i. Who is the actor(s) with the maximum number of movies ii. Who is the actor(s) with the highest average earnings per movie

Algorithm
Step-1 Import beautifulsoup pakage, import request,urlopen, urllib.request, re, os 
Step-2 Enter site url for web scraping
Step-3 Add actor name, actor id, average earningins per movie, number of movies and highest earning movie in text file 
Step-4 Determine actor with most number of movies 
Step-5 Determine actor with highest average earnings per movie 
Step-6 End


from bs4 import BeautifulSoup

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

import re
import os
#import regular impresions and operating system dependent funtotionality
constActorsDataSotred = "The data is stored in file actors.txt"
actorsHomeURL = "http://www.boxofficemojo.com/people/?view=Actor&pagenum=1&sort=sumgross&order=DESC&&p=.htm"

constFileName = "actors.txt"

urlBoxOfficeMojoHome = "http://www.boxofficemojo.com"

urlPre = "http://www.boxofficemojo.com/people/?view=Actor&pagenum="

urlPost = "&sort=sumgross&order=DESC&&p=.htm"
#enter url from where data needs to be scraped
# create text file where data needs to be saved
constPageHrefIncluded = "/people/?view=Actor&pagenum="

globalActorsWithMaxMovies= ''
globalMaxMovieCount = 0
globalMaxAvgEarnMovie = 0.0
globalActorsWithMaxAvgPerMovie = ''
globalRowIndex = 0
# define count for actor with max movies, average earnings and max average per movie
def removeChar(charToSub, a_strToStringChar):
  return re.sub(charToSub, '', a_strToStringChar)

def writeActorDataTable(a_strPageLink):

  request = requests.get(a_strPageLink)
  if request.status_code != 200:
    print("Error while retrieving page " + a_strPageLink)
    return

  urlSource = urlopen(a_strPageLink)
  currentPageSource = BeautifulSoup(urlSource.read())
  allTablesList = currentPageSource.findAll('table')
  actorTable = []

  tableDataList = []
  currentTableIndex = 0
  currentNestedTableIndex = 0
  
  isActorTableFound = False
  for currentTable in allTablesList:
    currentNestedTableList = currentTable.findAll('table')
    for nestedTable in currentNestedTableList:
      tableRowList = nestedTable.findAll('tr')
      if len(tableRowList) >= 1:
        tableDataList = tableRowList[0].findAll('td')
        for currentTableData in tableDataList:
          if currentTableData.get_text() == "Row":
            actorTable = nestedTable
            isActorTableFound = True
            break
      if isActorTableFound:
        break
    if isActorTableFound:
        break

  tableRows = actorTable.findAll('tr')
  currentRowIndex = 1
  rowCount = len(tableRows)

  global globalActorsWithMaxMovies
  global globalMaxMovieCount
  global globalMaxAvgEarnMovie
  global globalActorsWithMaxAvgPerMovie
  global globalRowIndex

  while currentRowIndex < rowCount:
    currentRow = tableRows[currentRowIndex]
    columns = currentRow.findAll('td')
    columnCount = len(columns)

    if columnCount == 7:
      globalRowIndex += 1
      rowNum = globalRowIndex
      actorName = columns[1].get_text()
#condition to separate all the required data collected on the text file by a ','
      actorId = columns[1].find('a').get('href')
      actorId = actorId.replace('.htm','')
      actorId = actorId.replace('./chart/?view=Actor&id=','')
#condition to put the actors ID in the 2nd column
      numOfMovies = columns[3].get_text()
      num1Picture = columns[5].get_text()
      totalGross = removeChar(',',columns[2].get_text())

      strAvgEarnPerMovie = columns[4].get_text()

      strAvgEarnPerMovie = strAvgEarnPerMovie.replace('$','')
      strAvgEarnPerMovie = strAvgEarnPerMovie.replace('k','')

      avgEarnPerMovie = float(strAvgEarnPerMovie)

      if int(numOfMovies) > globalMaxMovieCount:
        globalActorsWithMaxMovies = actorName
        globalMaxMovieCount = int(numOfMovies)
      elif numOfMovies == globalMaxMovieCount:
        globalActorsWithMaxMovies += ", "+ actorName

      if avgEarnPerMovie > globalMaxAvgEarnMovie:
        globalActorsWithMaxAvgPerMovie = actorName
        globalMaxAvgEarnMovie = avgEarnPerMovie
      elif avgEarnPerMovie == globalMaxAvgEarnMovie:
        if globalActorsWithMaxAvgPerMovie.find(actorName) == -1:
          globalActorsWithMaxAvgPerMovie += ", "+ actorName

      fileToWrite = open(constFileName, "a")
      fileToWrite.write(str(rowNum) +"," + actorName +","+ actorId +"," + totalGross +","+ numOfMovies +","+ num1Picture+"\n")

    currentRowIndex += 1
#condition of the order of the data of an actor to be written in the .text file




baseYearUrlToOpen = urlopen(actorsHomeURL)
readSource = BeautifulSoup(baseYearUrlToOpen.read())
#using beautifulsoup to read the web page of actors information
allPageList = []
allPageSet = []
#using append to add data at the end of the list
allLinks = readSource.findAll('a')
for hrefTag in allLinks:
  if str(hrefTag.get('href')).find(constPageHrefIncluded) != -1:
    allPageList.append(urlBoxOfficeMojoHome + str(hrefTag.get('href')))

allPageList.append(actorsHomeURL)

allPageSet = set(allPageList)

sortedPageList = []
pageCount = len(allPageSet) + 1
pageIndex = 1

while pageIndex <= pageCount:
  sortedPageList.append(urlPre + str(pageIndex) + urlPost)
  pageIndex += 1

if os.path.exists(constFileName):
  os.remove(constFileName)

globalActorsWithMaxMovies = ''
globalMaxMovieCount = 0
globalMaxAvgEarnMovie = 0.0
globalActorsWithMaxAvgPerMovie = ''
globalRowIndex = 0

for l_currentPage in sortedPageList:
  writeActorDataTable(l_currentPage)
print(constActorsDataSotred)

print("The actor(s) with the maximum number of movies is: "+ globalActorsWithMaxMovies);
print("The actor(s) with the highest average earnings per movie is: "+ globalActorsWithMaxAvgPerMovie)
